### 라이브러리 import

In [28]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

from joblib import dump

### 데이터 로드

In [29]:
# 데이터 로드
bank_df = pd.read_csv('../../data/BankChurners.csv')

### 전처리

In [30]:
# 필요없는 칼럼 제거
drop_columns = ['CLIENTNUM',
                'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
]
bank_df = bank_df.drop(columns=drop_columns)

bank_df = bank_df.loc[:, ~bank_df.columns.str.startswith("Total_")]

# 이탈여부 값 변환
bank_df['Attrition_Flag'] = bank_df['Attrition_Flag'].map({'Existing Customer': 0, 'Attrited Customer': 1})

### 원핫 인코딩

In [31]:
# 범주형 칼럼
cate_columns = ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']

# 원핫 인코딩
encoder = OneHotEncoder()
encoded_cate = encoder.fit_transform(bank_df[cate_columns]).toarray()
encoded_cate_df = pd.DataFrame(data=encoded_cate, columns=encoder.get_feature_names_out(cate_columns))

# 원래 데이터에서 범주형 칼럼 제거
bank_df = bank_df.drop(columns=cate_columns)

# 인코딩된 데이터와 결합
bank_df = pd.concat([bank_df, encoded_cate_df], axis=1)

# display(bank_df)

### 스케일링

In [32]:
# 스케일링 칼럼
scale_columns = ['Credit_Limit',  'Avg_Open_To_Buy']

# 스케일링링
scaler = StandardScaler()
bank_df[scale_columns] = scaler.fit_transform(bank_df[scale_columns])

### 학습 & 평가 데이터 분리

In [33]:
X = bank_df.drop(columns=['Attrition_Flag'])
y = bank_df['Attrition_Flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

### 모델 학습 및 평가

In [34]:
# 모델 선정
models = {
    # "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

# 모델 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)    
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # ROC-AUC 계산을 위한 확률값
    
    # 평가 지표 출력
    print(f"{name} ==========")
    print(f"Accuracy : {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision : {precision_score(y_test, y_pred):.4f}")
    print(f"Recall : {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score : {f1_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC : {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(f"\n>>>> Classification Report\n{classification_report(y_test, y_pred)}")

    # 특성 중요도 확인
    # if (name == 'Logistic Regression'):
    #     coef_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_[0]})
    #     print("\n>>>> Feature Coefficients\n", coef_importance.sort_values(by='Coefficient', ascending=False))
    # else:
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
    print("\n>>>> Feature Importance\n", feature_importance.sort_values(by='Importance', ascending=False))

    print("\n" + "="*100 + "\n")

Random Forest ==========
Accuracy : 0.8569
Precision : 0.7108
Recall : 0.1815
F1 Score : 0.2892
ROC-AUC : 0.8317

>>>> Classification Report
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1701
           1       0.71      0.18      0.29       325

    accuracy                           0.86      2026
   macro avg       0.79      0.58      0.60      2026
weighted avg       0.84      0.86      0.82      2026


>>>> Feature Importance
                            Feature  Importance
7            Avg_Utilization_Ratio    0.156690
6                  Avg_Open_To_Buy    0.128866
5                     Credit_Limit    0.126652
0                     Customer_Age    0.095315
2                   Months_on_book    0.092308
4            Contacts_Count_12_mon    0.078485
3           Months_Inactive_12_mon    0.065984
1                  Dependent_count    0.050078
12        Education_Level_Graduate    0.015920
19           Marital_Status_Single  

- ### 오버 샘플링 및 하이퍼 파라미터 튜닝

In [ ]:
# 오버샘플링 적용
smote = SMOTE(random_state=42)
X_train_resample, y_train_resample = smote.fit_resample(X_train, y_train)

# 모델 선정
models = {
    # "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

# 파라미터 설정
param_grids = {
    # "Logistic Regression": {
    #     'C': [0.01, 0.1, 1, 10],              # 규제 강도
    #     'penalty': ['l1', 'l2'],               # 규제 유형
    #     'solver': ['liblinear']                # l1과 l2 모두 지원하는 solver
    # },
    "Random Forest": {
        'n_estimators': [50, 100, 200],                 # 트리 개수
        'max_depth': [3, 5],                          # 최대 깊이
        'min_samples_split': [2, 5, 10],                # 노드 분할 최소 샘플 : 값이 클수록 트리가 덜 복잡해져 과적합을 줄이는 효과
        'min_samples_leaf': [1, 2, 4],                  # 리프 노드 최소 샘플 : 값이 크면 모델이 단순 (클래스 불균형이 심하면 크게 설정)
        # 'max_features': ['sqrt', 'log2', 0.3, 0.5],     # 특성 샘플링 비율 : 각 트리에서 사용할 특성의 최대 개수 (무작위성을 높여 모델의 다양성을 증가)
        # 'class_weight' : ['balanced']                   # 클래스 가중치 : 클래스 불균형을 해결하기 위해 클래스에 가중치를 부여
    },
    "XGBoost": {
        'n_estimators': [50, 100, 200],                 # 트리 개수
        'max_depth': [3, 5],                        # 최대 깊이 : XGBoost는 깊이가 얕아도 잘 작동한다!
        'learning_rate': [0.01, 0.05, 0.1, 0.3],        # 학습률
        'subsample': [0.6, 0.8, 1.0],                   # 각 트리 학습에 사용할 데이터 샘플 비율 : 값이 낮을수록 과적합 방지
    },
    "LightGBM": {
        'n_estimators': [50, 100, 200],
        'max_depth': [2, 5],  
        'learning_rate': [0.01, 0.05, 0.1, 0.3],
        'num_leaves': [20, 31, 50],                     # 한 트리의 최대 리프 노드 수 : 2^(max_depth)보다 작아야 과적합을 줄이는 데 유리
        'reg_lambda' : [0.1, 1.0]                       # L2 규제 : 과적합을 방지하고 모델을 안정화
    },
}

# 모델 학습 및 평가
for name, model in models.items():
    # 파라미터 학습
    print(f"\nGridSearchCV Search Best Params for {name}..............................")
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[name], cv=5, scoring='f1', n_jobs=-1, verbose=1)
    grid_search.fit(X_train_resample, y_train_resample)
    
    # 최적 모델 선정
    best_model = grid_search.best_estimator_
    print(f">>>> Best Parameters for {name}\n{grid_search.best_params_}")
    
    # 교차 검증
    print(f"Cross Val Score : {cross_val_score(best_model, X_train_resample, y_train_resample, scoring='f1', cv=5)}")

    # 예측
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # ROC-AUC 계산을 위한 확률값
    
    # 평가 지표 출력
    print(f"{name} ==========")
    print(f"Accuracy : {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision : {precision_score(y_test, y_pred):.4f}")
    print(f"Recall : {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score : {f1_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC : {roc_auc_score(y_test, y_pred_proba):.4f}")    
    print(f"\n>>>> Classification Report\n{classification_report(y_test, y_pred)}")

    # 특성 중요도 확인
    # if (name == "Logistic Regression"):
    #     coef_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': best_model.coef_[0]})
    #     print("\n>>>> Feature Coefficients\n", coef_importance.sort_values(by='Coefficient', ascending=False))
    # else:
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': best_model.feature_importances_})
    print("\n>>>> Feature Importance\n", feature_importance.sort_values(by='Importance', ascending=False))

    print("\n" + "="*100 + "\n")


GridSearchCV Search Best Params for Random Forest..............................
Fitting 5 folds for each of 54 candidates, totalling 270 fits
>>>> Best Parameters for Random Forest
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Cross Val Score : [0.53618261 0.86303806 0.84566145 0.85529623 0.85008818]
Random Forest ==========
Accuracy : 0.7789
Precision : 0.3705
Recall : 0.5415
F1 Score : 0.4400
ROC-AUC : 0.7688

>>>> Classification Report
              precision    recall  f1-score   support

           0       0.90      0.82      0.86      1701
           1       0.37      0.54      0.44       325

    accuracy                           0.78      2026
   macro avg       0.64      0.68      0.65      2026
weighted avg       0.82      0.78      0.79      2026


>>>> Feature Importance
                            Feature  Importance
7            Avg_Utilization_Ratio    0.162791
3           Months_Inactive_12_mon    0.118235
13     Education_Level_

### 최종 모델 선정 및 저장

In [23]:
# dump(model, "model.joblib")